In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
env_name = 'QuadruppedWalk-v1'#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
a2c_config = games_configurations.quadrupped_lstm_config
#a2c_config = games_configurations.roboschoolhumanoid_config
#a2c_config = games_configurations.bipedalwalker_lstm_config
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)


robot lib not fouund
Box(42,)
Box(12,)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-07-20 15:55:54,140	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_15-55-54_13531/logs.
2019-07-20 15:55:54,256	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:31657 to respond...
2019-07-20 15:55:54,403	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:45014 to respond...
2019-07-20 15:55:54,410	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-07-20 15:55:54,470	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-20_15-55-54_13531/logs.
2019-07-20 15:55:54,472	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.106',
 'redis_address': '192.168.3.106:31657',
 'object_store_address': '/tmp/ray/session_2019-07-20_15-55-54_13531/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-07-20_15-55-54_13531/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks



agent = A2CAgent(sess,'run', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest_run')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
0
1
2
3


2019-07-20 15:55:55,762	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


4
5
6
7
0
Instructions for updating:
Use tf.cast instead.
(pid=13631) 
(pid=13631) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=13631) For more information, please see:
(pid=13631)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=13631)   * https://github.com/tensorflow/addons
(pid=13631) If you depend on functionality not listed there, please file an issue.
(pid=13631) 
(pid=13624) 
(pid=13624) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=13624) For more information, please see:
(pid=13624)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=13624)   * https://github.com/tensorflow/addons
(pid=13624) If you depend on functionality not listed there, please file an issue.
(pid=13624) 
(pid=13631) robot lib not fouund
(pid=13624) robot lib not fouund
(pid=13629) 
(pid=13629) WARNING: The TensorFlow contrib module will not be in

Frames per seconds:  1014.8900019813544
saving next best rewards:  308.79006368561653
Frames per seconds:  1141.1016233788662
saving next best rewards:  424.04822145125706
Frames per seconds:  1132.7245618158809
saving next best rewards:  504.64721622236175
Frames per seconds:  1106.4368247133757
saving next best rewards:  531.9320809565261
Frames per seconds:  1094.606947990865
saving next best rewards:  671.2519523762807
Frames per seconds:  1030.058090442427
saving next best rewards:  775.8524993404102
Frames per seconds:  965.447136028154
saving next best rewards:  886.7190039406415
Frames per seconds:  1073.7000168055956
saving next best rewards:  965.9117932829633
Frames per seconds:  1084.0218060407733
saving next best rewards:  992.8724901854198
Frames per seconds:  988.3066659548657
Frames per seconds:  1100.2740384649255
Frames per seconds:  1127.8476356837953
Frames per seconds:  1152.1475532672787
Frames per seconds:  1141.9040007688968
Frames per seconds:  1094.46141506784

Frames per seconds:  1093.29506457522
Frames per seconds:  1078.739556946109
Frames per seconds:  1071.7452964808183
Frames per seconds:  1115.3433647049308
Frames per seconds:  1095.3849184496296
Frames per seconds:  1109.9853429131354
Frames per seconds:  1117.3788916934864
Frames per seconds:  1111.0018398925515
Frames per seconds:  1110.6124928436707
Frames per seconds:  1115.9488218595643
Frames per seconds:  1048.9172950500435
Frames per seconds:  1120.2664913873152
Frames per seconds:  1107.4248427322275
Frames per seconds:  1095.8368397508846
Frames per seconds:  1110.9598108844884
Frames per seconds:  1083.891109375414
Frames per seconds:  1086.680109048357
Frames per seconds:  1073.9187521144108
Frames per seconds:  1071.4884168912517
Frames per seconds:  1060.8409373607558
Frames per seconds:  1100.6692147972328
Frames per seconds:  1104.9100236875734
Frames per seconds:  1098.6602627544685
Frames per seconds:  1117.2283307590715
Frames per seconds:  1087.267362029436
Frames

Frames per seconds:  1051.6414475250795
Frames per seconds:  1126.2148485544155
Frames per seconds:  1099.985202205135
Frames per seconds:  1090.7353443226284
Frames per seconds:  1124.720924629769
Frames per seconds:  1093.7519475203887
Frames per seconds:  1102.1433698001435
Frames per seconds:  1100.3931395153481
Frames per seconds:  1062.23700060569
Frames per seconds:  1091.905189112364
Frames per seconds:  1071.8570972765763
Frames per seconds:  1098.6820437278784
Frames per seconds:  1094.2060763675038
Frames per seconds:  1080.2239780362008
Frames per seconds:  1112.1461777743598
Frames per seconds:  1099.2051869571594
Frames per seconds:  989.1238261765726
Frames per seconds:  1065.9872993905833
Frames per seconds:  1043.3123795179094
Frames per seconds:  1058.1916318332337
Frames per seconds:  1104.988054653717
Frames per seconds:  1107.0725970638769
Frames per seconds:  1100.1081151969247
Frames per seconds:  1103.0174230568864
Frames per seconds:  1057.6571069250226
Frames 

Frames per seconds:  1109.110240457964
Frames per seconds:  1085.2160849704135
Frames per seconds:  1025.7393997862291
Frames per seconds:  1107.510083317883
Frames per seconds:  1077.143307923509
Frames per seconds:  1109.6776958575053
Frames per seconds:  1101.9922917682152
Frames per seconds:  1089.6850053127305
Frames per seconds:  1110.998103851953
Frames per seconds:  1123.6563233685144
Frames per seconds:  1043.8873630882385
Frames per seconds:  1108.8931834693878
Frames per seconds:  1109.9363632070222
Frames per seconds:  1076.7825232478365
Frames per seconds:  1080.10349859378
Frames per seconds:  1108.3745095663512
Frames per seconds:  1092.98936674844
Frames per seconds:  1076.8803244004448
Frames per seconds:  1008.2031970965405
Frames per seconds:  1079.4589760681872
Frames per seconds:  1108.684224710191
Frames per seconds:  1112.7135016829434
Frames per seconds:  1092.5241579089718
Frames per seconds:  1099.4960061367449
Frames per seconds:  1093.0418692303508
Frames pe

Frames per seconds:  1102.796372940586
Frames per seconds:  1075.0576946182127
Frames per seconds:  1112.0194085759356
Frames per seconds:  1035.5570805458572
Frames per seconds:  1083.0569941333547
Frames per seconds:  1107.354318566477
Frames per seconds:  1118.5374304668737
Frames per seconds:  1092.6358886558664
Frames per seconds:  1107.130599421697
Frames per seconds:  1112.8657316623205
Frames per seconds:  1086.1953936239593
Frames per seconds:  1040.9755568837486
Frames per seconds:  1105.7180141436443
Frames per seconds:  1099.7684633038461
Frames per seconds:  1099.9168195336472
Frames per seconds:  1105.6770955507936
Frames per seconds:  1088.3590989002232
Frames per seconds:  1096.6759921434505
Frames per seconds:  1102.8077702193289
Frames per seconds:  1030.4070261099212
Frames per seconds:  1096.565183445646
Frames per seconds:  1113.0877362778335
Frames per seconds:  1093.4915108641565
Frames per seconds:  1116.0084105320816
Frames per seconds:  1105.76669342282
Frames

Frames per seconds:  1091.7735560517508
Frames per seconds:  1110.5455104433768
Frames per seconds:  1036.059123363076
Frames per seconds:  1098.9410927267713
Frames per seconds:  1102.3657143429139
Frames per seconds:  1116.073298379661
Frames per seconds:  1088.7091938642723
Frames per seconds:  1106.9912042500023
Frames per seconds:  1111.1870927974412
Frames per seconds:  1091.8346848015137
Frames per seconds:  1042.8353771059228
Frames per seconds:  1109.9938054379256
Frames per seconds:  1080.5224399741
Frames per seconds:  1102.5523436539352
Frames per seconds:  1105.17699492882
Frames per seconds:  1080.444496489535
Frames per seconds:  1110.8867526392653
Frames per seconds:  1117.4040375796635
Frames per seconds:  1029.9940498505327
Frames per seconds:  1108.2733353146505
Frames per seconds:  1118.6875431761005
Frames per seconds:  1081.5564070271485
Frames per seconds:  1099.900834285617
Frames per seconds:  1098.3696045698662
Frames per seconds:  1079.1529667635943
Frames pe

RayTaskError: [36mray_RayWorker:step()[39m (pid=13748, host=trrrrr-MS-7A71)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/memory_monitor.py", line 77, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node trrrrr-MS-7A71 is used (32.06 / 33.69 GB). The top 5 memory consumers are:

PID	MEM	COMMAND
13792	1.74GB	ray_RayWorker
13628	1.74GB	ray_RayWorker
13769	1.74GB	ray_RayWorker
13727	1.74GB	ray_RayWorker
13748	1.74GB	ray_RayWorker:step()

In addition, ~0.34 GB of shared memory is currently being used by the Ray object store. You can set the object store size with the `object_store_memory` parameter when starting Ray, and the max Redis size with `redis_max_memory`.


In [6]:
#agent.save('nn/latest_run')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()